# Abstract 

La posibilidad que brindan los sistemas automatizados de decision y clasificacion son sobresalientes al producir mejoras de producto y reducir costos o aumentar ganacias, de manera sistematica y eficaz. Muchas empresas relacionadas al juego y apuestas utilizan esto para predecir numeros ante la gran cantidad de datos que se nos brinda dados en el dataset. Asi mismo , el cliente propone una solucion practica ante las circunstacias economicas que atraviesa.

En base a esta informacion , se puede hacer una investigacion y resolver las practicas beneficiosas de los jugadores y del equipo en cada uno de los juegos, es decir , es posible hacer preguntas y responderlas para obtener nuevos conocimientos sobre los equipos, los jugadores y la competencia en sí. Por ejemplo, se puede analizar la tendencia de los equipos en casa y fuera de casa, las estadísticas de los jugadores en diferentes momentos del partido, o el porcentaje de victorias y derrotas de los equipos.

Este proyecto se basara en utilizar los datos disponibles para encontrar las diferencias y probrabilidades en cada partido dado durante los años jugados por la NBA. El objetivo se centrara en poder adquirir una variable que determine un numero en el cual se basara la cuota cobrada por el sistema de apuestas. Ademas , este analisis proovera al cliente con el gran beneficio del conocimiento y la ventaja por sobre la competencia , o hacia los mismos usuarios de la plataforma.

Asi, lo visto durante un analisis exploratorio de los datos y las caracteristicas del conjunto , se plantearan las primeras hipotesis para luego establecer un modelo predictivo de los datos de interes. Luego de un resultado final , se podra verificar el uso de las herramientas y las capacidades un sistema de ML.

## Analisis exploratorio complementario (Adquisicion de datos por APIs)

Para verificar con mayor veracidad los datos , podria utilizarse alguna API que nos permita obtener datos actualizados acerca de los juegos que no se encuentran en el dataset. Estos datos nuevos podrian utilizarse en la parte de validacion del modelo o para sacar nuevas hipotesis en base a las tendencias de los nuevos juegos. 

Hay que comentar que estos tipos de datos suelen ser de pago y no siempre se encuentran de manera libre o por "suscripcion gratuita". Lo ideal seria utilizar APIs que sean oficiales , limpias de datos erroneos o ordenados de manera correcta y sistematica.

En esta oportunidad utilizaremos la API publica brindada por *"Ball don't lie"* , que a pesar de ser una API NO OFICIAL, nos permitira obtener algunos datos relevantes de las temporadas que no tenemos en el dataset de nuestro proyecto. Su website es "https://www.balldontlie.io/home.html#introduction" en la que se encuentra la manera de realizar "querys" a su url.

Recuerde que los datos del dataset provisto durante las etapas anteriores , alcanzan las temporadas 2003-2021 (Ultima fecha registrada = 2022-03-12)

Como se nombro anteriormente , esta informacion se utilizara para complementar los datos de las temporadas anteriores , por lo tanto , se procedera a realizar un "GET" para la informacion de la temporadara ("SEASON") 2022.

In [1]:
import requests

url = "https://www.balldontlie.io/api/v1/games?seasons[]=2022&per_page=100"

response = requests.get(url)
data = response.json()
data['meta']

{'total_pages': 14,
 'current_page': 1,
 'next_page': 2,
 'per_page': 100,
 'total_count': 1320}

Al ser demasiada informacion para procesar , utilizan un sistema de paginas que proveen la informacion por "bloques". En el apartado ['meta'] , nos provee de la informacion que se obtiene del uso de la API , que nos informa que existen **16 paginas** y **1307 claves** para esta temporada.

Se utilizara 100 resultados por pagina ya que es el limite que dispone la documentacion de la API y de esta manera , sera necesario acceder a las 14 paginas para completar el DF con todos los datos. Una manera de hacerlo es iterar sobre la utilizacion de la API y asi obtener la informacion completa.

In [2]:
import pandas as pd

#lista para almacenar los diccionarios de estadísticas
stats_list = []

for i in range (1,15):
    #global stats_list
    url = 'https://www.balldontlie.io/api/v1/games?seasons[]=2022&per_page=100&page=' + str(i)
    response = requests.get(url)
    data = response.json()
    # Iterar sobre cada partido en la pagina y agregar las estadísticas como columnas en el DataFrame
    for game in data['data']:
        stats = {'GAME_ID': game['id'],
                'GAME_DATE_EST' : game['date'],
                'TEAM_ID_home' : game['home_team']['id'],
                'PTS_home' : game['home_team_score'],
                'SEASON' : game['season'],
                'TEAM_ID_away' : game['visitor_team']['id'],
                'PTS_away' : game['visitor_team_score']}
        stats_list.append(stats)
    
df = pd.DataFrame(stats_list)
df

,GAME_ID,GAME_DATE_EST,TEAM_ID_home,PTS_home,SEASON,TEAM_ID_away,PTS_away
0,857369,2022-10-20T00:00:00.000Z,14,97,2022,13,103
1,857370,2022-10-20T00:00:00.000Z,23,88,2022,17,90
2,857381,2022-10-21T00:00:00.000Z,25,113,2022,24,111
3,857374,2022-10-21T00:00:00.000Z,30,102,2022,5,100
4,857376,2022-10-21T00:00:00.000Z,11,122,2022,15,129
...,...,...,...,...,...,...,...
1315,1007821,2023-05-22T00:00:00.000Z,14,111,2022,8,113
1316,1007822,2023-05-23T00:00:00.000Z,16,99,2022,2,116
1317,1007824,2023-05-25T00:00:00.000Z,2,110,2022,16,97
1318,1007826,2023-05-27T00:00:00.000Z,16,103,2022,2,104


Como se puede ver , el DF generado contiene los registros de toda la temporada 2022.

# Data Wrangling en datos actualizados

Se hara una limpieza y orden de los datos recientemente adquiridos para la futura utilizacion.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   GAME_ID        1320 non-null   int64 
 1   GAME_DATE_EST  1320 non-null   object
 2   TEAM_ID_home   1320 non-null   int64 
 3   PTS_home       1320 non-null   int64 
 4   SEASON         1320 non-null   int64 
 5   TEAM_ID_away   1320 non-null   int64 
 6   PTS_away       1320 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 72.3+ KB


Como condicion general , se busca que los datos numericos sean del tipo *float* o *int* .Tampoco se encontraron valores nulos entre los datos pero existe la posibilidad de que haya datos invalidos. Para eso se usara el metodo ".describe()" que realizara un analisis rapido estadisitico y asi saber si algun dato es incorrecto.

In [4]:
df.describe()

,GAME_ID,TEAM_ID_home,PTS_home,SEASON,TEAM_ID_away,PTS_away
count,1.320000e+03,1320.000000,1320.000000,1320.0,1320.000000,1320.000000
mean,8.633633e+05,15.353030,115.630303,2022.0,15.352273,113.030303
std,2.349893e+04,8.603184,11.991075,0.0,8.589413,12.001920
min,8.573550e+05,1.000000,80.000000,2022.0,1.000000,79.000000
25%,8.576848e+05,8.000000,108.000000,2022.0,8.000000,105.000000
50%,8.580145e+05,15.000000,116.000000,2022.0,15.000000,113.000000
75%,8.583442e+05,23.000000,124.000000,2022.0,23.000000,121.000000
max,1.011169e+06,30.000000,175.000000,2022.0,30.000000,176.000000


A pesar de que hay posibilidad de que existan partidos en los que haya 0 anotaciones (PTS_home , PTS_away) , es sumamente improbable. Por lo tanto se tomara las filas estas como **datos erroneos o faltantes**. Se contaran cuantos registros cumplen con estas condiciones

In [5]:
# Filtrar las filas que cumplen con la condición de que "PTS_home" sea igual a 0 o "PTS_away" sea igual a 0
filtered_df = df.loc[(df['PTS_home'] == 0) | (df['PTS_away'] == 0)]
filtered_df


,GAME_ID,GAME_DATE_EST,TEAM_ID_home,PTS_home,SEASON,TEAM_ID_away,PTS_away


Como se puede ver , los casos que son nulos son pocos y no afectan realmente a los datos , por lo tanto , se procedera con la eliminacion de estos registros. El siguiente codigo es una manera distinta de eliminacion de los registros aunque igual de eficaz

In [6]:
df = df.loc[(df['PTS_home'] != 0) | (df['PTS_away'] != 0)]
df.describe()

,GAME_ID,TEAM_ID_home,PTS_home,SEASON,TEAM_ID_away,PTS_away
count,1.320000e+03,1320.000000,1320.000000,1320.0,1320.000000,1320.000000
mean,8.633633e+05,15.353030,115.630303,2022.0,15.352273,113.030303
std,2.349893e+04,8.603184,11.991075,0.0,8.589413,12.001920
min,8.573550e+05,1.000000,80.000000,2022.0,1.000000,79.000000
25%,8.576848e+05,8.000000,108.000000,2022.0,8.000000,105.000000
50%,8.580145e+05,15.000000,116.000000,2022.0,15.000000,113.000000
75%,8.583442e+05,23.000000,124.000000,2022.0,23.000000,121.000000
max,1.011169e+06,30.000000,175.000000,2022.0,30.000000,176.000000


Pasaremos a identificar casos duplicados, por ejemplo, partidos anotados dos veces. Una manera podria ser analizar la fecha de los partidos para ver si existen dos registros iguales en la columna *"GAME_DATE_EST"*

In [7]:
#Realizamos la suma de las filas que estan duplicadas (True)
df_dup=sum(df.duplicated('GAME_DATE_EST'))
df_dup

1108

A simple vista puede notarse que existen una gran cantidad de duplicados, demasiada representativa a la muestra total de los datos. Sera necesario distinguirlos con distinto criterio.

In [8]:
df

,GAME_ID,GAME_DATE_EST,TEAM_ID_home,PTS_home,SEASON,TEAM_ID_away,PTS_away
0,857369,2022-10-20T00:00:00.000Z,14,97,2022,13,103
1,857370,2022-10-20T00:00:00.000Z,23,88,2022,17,90
2,857381,2022-10-21T00:00:00.000Z,25,113,2022,24,111
3,857374,2022-10-21T00:00:00.000Z,30,102,2022,5,100
4,857376,2022-10-21T00:00:00.000Z,11,122,2022,15,129
...,...,...,...,...,...,...,...
1315,1007821,2023-05-22T00:00:00.000Z,14,111,2022,8,113
1316,1007822,2023-05-23T00:00:00.000Z,16,99,2022,2,116
1317,1007824,2023-05-25T00:00:00.000Z,2,110,2022,16,97
1318,1007826,2023-05-27T00:00:00.000Z,16,103,2022,2,104


In [9]:
#Realizamos la suma de las filas que estan duplicadas usando "GAME_ID" (True)
df_dup=sum(df.duplicated('GAME_ID'))
df_dup

0

Asi se puede verificar que el error se debe al problema en la anotacion de la fecha en "GAME_DATE_EST" y no es correcto decir que existen duplicados ya que "GAME_ID" demuestra que no hay repetidos

Este problema en la fecha puede traer confusion , por lo tanto , seria una buena conducta corregir el formato del dato a uno que provea informacion real. Es decir , se corregira el dato del horario ya que en todos los registros son **'T00:00:00.000Z'**.

Se usara el metodo **".apply"** en el df que almacena los datos para ejecutar una funcion que realice esta accion registro por registro.

In [10]:
# Función para eliminar el horario y dejar solo la fecha
def eliminar_horario(fecha):
    fecha_dt = pd.to_datetime(fecha)
    return fecha_dt.strftime('%Y-%m-%d')

# Aplicar la función a la columna 'fechas' utilizando apply
df['GAME_DATE_EST'] = df['GAME_DATE_EST'].apply(eliminar_horario)
df

,GAME_ID,GAME_DATE_EST,TEAM_ID_home,PTS_home,SEASON,TEAM_ID_away,PTS_away
0,857369,2022-10-20,14,97,2022,13,103
1,857370,2022-10-20,23,88,2022,17,90
2,857381,2022-10-21,25,113,2022,24,111
3,857374,2022-10-21,30,102,2022,5,100
4,857376,2022-10-21,11,122,2022,15,129
...,...,...,...,...,...,...,...
1315,1007821,2023-05-22,14,111,2022,8,113
1316,1007822,2023-05-23,16,99,2022,2,116
1317,1007824,2023-05-25,2,110,2022,16,97
1318,1007826,2023-05-27,16,103,2022,2,104


De esta manera , los valores en "GAME_DATE_EST" pasaran a ser el dia en el que se realizo el partido

Tambien nos interesa que sea un conjunto de datos ordenados (a pesar de estar relativamente ordenados , no siguen un orden completo) , asi , se buscara de ordenarlos por el "GAME_ID" mostrando un orden especifico.

Para esta tarea se usara el metodo ".sort_values()" , que permitira ordenar las filas del dataframe de manera segun la caracteristica que consideremos apropiada ("GAME_ID")

In [11]:
df = df.sort_values(by=['GAME_ID'])
df

,GAME_ID,GAME_DATE_EST,TEAM_ID_home,PTS_home,SEASON,TEAM_ID_away,PTS_away
1069,857355,2022-10-18,2,126,2022,23,117
1073,857356,2022-10-18,10,123,2022,14,109
1016,857357,2022-10-19,12,107,2022,30,114
1038,857358,2022-10-19,9,113,2022,22,109
953,857359,2022-10-19,3,108,2022,19,130
...,...,...,...,...,...,...,...
1298,1011165,2023-06-01,8,104,2022,16,93
1299,1011166,2023-06-04,8,108,2022,16,111
1300,1011167,2023-06-07,16,94,2022,8,109
1301,1011168,2023-06-09,16,95,2022,8,108


Como verificacion , se puede observar las diferencias que existen en el orden del indice del df. 

Como ultimo paso antes de exportarlo , se reordenara el indice para continuar con el correcto orden del DF

In [12]:
df = df.reset_index(drop=True)
df

,GAME_ID,GAME_DATE_EST,TEAM_ID_home,PTS_home,SEASON,TEAM_ID_away,PTS_away
0,857355,2022-10-18,2,126,2022,23,117
1,857356,2022-10-18,10,123,2022,14,109
2,857357,2022-10-19,12,107,2022,30,114
3,857358,2022-10-19,9,113,2022,22,109
4,857359,2022-10-19,3,108,2022,19,130
...,...,...,...,...,...,...,...
1315,1011165,2023-06-01,8,104,2022,16,93
1316,1011166,2023-06-04,8,108,2022,16,111
1317,1011167,2023-06-07,16,94,2022,8,109
1318,1011168,2023-06-09,16,95,2022,8,108


Finalmente, el Dataframe ya posee datos utlizables y accesibles de manera confiable. 

Para poder utilizar estos datos sera necesario exportarlos en cualquiera de los formatos que sea conveniente. Continuando con los formatos anteriores, se exportara a un formato ".csv" para la posterior lectura del mismo.

In [13]:
# Exportar el DataFrame a un archivo CSV
df.to_csv('games_2022.csv', index=False)

Ahora que ya es un DataSet completo , debe ser necesario la descripcion del mismo:

## Introduccion de DataSet (games_2022.csv)

Este DataSet contiene los datos de los juegos de la NBA en la ultima temporada (2022). Este posee 1299 filas de datos y 7 columnas con los siguientes datos:

GAME_ID: Identificación única del juego.

GAME_DATE_EST: Fecha del juego en formato "YYYY-MM-DD".

TEAM_ID_home: ID del equipo local (equipo que juega en su propia cancha).

PTS_home: Puntuación del equipo local en ese juego.

SEASON: Temporada del juego.

TEAM_ID_away: ID del equipo visitante (equipo que juega fuera de casa).

PTS_away: Puntuación del equipo visitante en ese juego.

Es posible que se desee guardar la descripcion rapida del .csv para un rapido acceso a la descripcion del mismo.